In [1]:
from keras.layers import Input, Lambda, Dense, Flatten,Dropout
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from keras.applications.efficientnet import EfficientNetB1
from keras.applications.efficientnet import preprocess_input
import numpy as np
import math
from numpy import save,load
from PIL import Image
from pathlib import Path
import os
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
Path("/content/dataset/train/male").mkdir(parents=True, exist_ok=True)
Path("/content/dataset/test/male").mkdir(parents=True, exist_ok=True)
Path("/content/dataset/train/female").mkdir(parents=True, exist_ok=True)
Path("/content/dataset/test/female").mkdir(parents=True, exist_ok=True)
Path("/content/dataset_2").mkdir(parents=True, exist_ok=True)

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/Colab Notebooks/NewTrain4K/', output="/content/dataset_2/", seed=1337, ratio=(.8,0.1,0.1)) 

In [ ]:
! ls /content/dataset_2/train/Male | wc -l
! ls /content/dataset_2/test/Male | wc -l

! ls /content/dataset_2/train/Female | wc -l
! ls /content/dataset_2/test/Female | wc -l

In [ ]:
train_data_dir = '/content/dataset_2/train'
test_data_dir = '/content/dataset_2/test'
BATCH_SIZE=8
class_subset = sorted(os.listdir('/content/dataset_2/test'))[:10] # Using only the first 10 classes
train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) # efficientnet preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # efficientnet preprocessing

traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(128, 128),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(128, 128),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)
print(class_subset)

In [ ]:
def create_efficientb1_Based_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
  conv_base = EfficientNetB1(
                    include_top=True,
                    weights="imagenet",
                    input_tensor=None,
                    input_shape=None,
                    pooling=None,
                    classes=1000,
                    classifier_activation="softmax",
                )
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    model = Model(inputs=conv_base.input, outputs=output_layer)

    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
input_shape = (256, 256, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=2

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

res_model = create_efficientb1_Based_model(input_shape, n_classes, optim_1, fine_tune=0)

In [ ]:
! pip install livelossplot

In [ ]:
from livelossplot.inputs.keras import PlotLossesCallback

plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='/content/drive/MyDrive/ENEL645/savedmodels/efficientnet_model_v14K.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
%%time 

efficientnet_history = res_model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[tl_checkpoint_1, early_stop, plot_loss_1],
                            verbose=1)


In [ ]:
from keras.preprocessing import image
model = tf.keras.models.load_model('/content/drive/MyDrive/ENEL645/savedmodels/efficientnet_model_v14K.weights.best.hdf5')
img_width, img_height = 240,240
model.summary()

In [ ]:
import matplotlib.pyplot as plt
img = image.load_img('/content/drive/MyDrive/Colab Notebooks/NewTrain/Male/0 (1).jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])
classes = model.predict(images)
classes=np.argmax(classes,1)
print("Male Image")
print("female:"+str( len(np.where( classes == 0)[0])))
print("male:"+str( len(np.where( classes == 1)[0])))
plt.imshow(img, cmap = "gray")
